In [1]:
import sqlite3
from threading import Thread
import time

In [2]:
# Connexion à la base de données SQLite pour l'initialisation
init_conn = sqlite3.connect('example.db')
init_cursor = init_conn.cursor()

# Création de la table
init_cursor.execute('''
DROP TABLE IF  EXISTS comptes
''')

init_cursor.execute('''
CREATE TABLE IF NOT EXISTS comptes (
    id INTEGER PRIMARY KEY,
    solde INTEGER
)
''')

# Initialiser deux comptes pour le test
init_cursor.execute("INSERT INTO comptes (solde) VALUES (500)")
init_cursor.execute("INSERT INTO comptes (solde) VALUES (500)")
init_conn.commit()
init_conn.close()


In [3]:
def transaction_bancaire(compte_source, compte_dest, montant):
    # Créer une nouvelle connexion et un curseur pour ce thread
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    
    try:
        cursor.execute("BEGIN")
        
        # Récupérer le solde du compte source
        cursor.execute("SELECT solde FROM comptes WHERE id = ?", (compte_source,))
        solde_source = cursor.fetchone()[0]
        
        # Vérifier si le compte source a suffisamment de fonds
        if solde_source >= montant:
            # Débiter le compte source
            cursor.execute("UPDATE comptes SET solde = solde - ? WHERE id = ?", (montant, compte_source))
            # Simuler un délai pour accentuer les problèmes de concurrence
            time.sleep(0.1)
            # Créditer le compte destinataire
            cursor.execute("UPDATE comptes SET solde = solde + ? WHERE id = ?", (montant, compte_dest))
        else:
            print(f"Compte {compte_source} n'a pas assez de fonds.")
        
        conn.commit()
        print(f"Transféré {montant} du compte {compte_source} au compte {compte_dest}")
    except Exception as e:
        conn.rollback()
        print(f"Transaction échouée: {e}")
    finally:
        conn.close()


In [4]:
def concurrent_transactions():
    thread1 = Thread(target=transaction_bancaire, args=(1, 2, 100))
    thread2 = Thread(target=transaction_bancaire, args=(1, 2, 200))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()

# Appel de la fonction pour simuler les transactions concurrentes
concurrent_transactions()

Transaction échouée: database is locked
Transféré 100 du compte 1 au compte 2


In [5]:
def verification_soldes():
    conn = sqlite3.connect('example.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM comptes")
    comptes = cursor.fetchall()
    for compte in comptes:
        print(f"Compte {compte[0]}: Solde = {compte[1]}")
    conn.close()

# Appel de la fonction pour vérifier les soldes
verification_soldes()

Compte 1: Solde = 400
Compte 2: Solde = 600


### Interprétation des résultats

 **Thread 1 ou Thread 2 peut obtenir le verrou en premier** :
   - **Thread 1** ou **Thread 2** peut obtenir le verrou pour effectuer sa transaction avant l'autre. Cela dépend de facteurs tels que l'ordre d'exécution et la gestion des verrous par le système d'exploitation et SQLite.

 **Transaction réussie pour le thread qui obtient le verrou** :
   - Le thread qui obtient le verrou en premier peut effectuer ses modifications sans interruption et compléter sa transaction.
   - **Exemple** : Si **Thread 2** obtient le verrou en premier, il transfère 200 unités du compte 1 au compte 2. Si **Thread 1** obtient le verrou en premier, il transfère 100 unités.

 **Échec de la transaction pour le thread qui n'obtient pas le verrou** :
   - Le thread qui tente d'accéder à la base de données pendant que l'autre thread a verrouillé la base de données échoue avec l'erreur "database is locked".

### Pourquoi cela s'est-il produit ?

- **Ordre d'exécution et accès aux verrous** : Bien que les threads soient lancés presque simultanément, le système d'exploitation et la gestion des verrous  déterminent quel thread obtient le verrou en premier. Cela signifie qu'un thread peut obtenir le verrou et commencer sa transaction avant l'autre.
- **Verrouillage de la base de données** : Une fois qu'un thread a obtenu le verrou, il peut effectuer ses modifications sans interruption. Le verrou empêche d'autres threads de modifier la base de données simultanément.
- **Échec du thread concurrent** : Le thread qui n'obtient pas le verrou en premier tente d'accéder à la base de données pendant qu'elle est verrouillée, entraînant l'échec de la transaction avec l'erreur "database is locked".